In [ ]:
!pip install gradio transformers sumy torch nltk evaluate rouge-score bert_score -q


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   

In [ ]:
!pip install gradio transformers sumy torch nltk -q


In [ ]:
import torch
import nltk
nltk.download('punkt')

import gradio as gr
from transformers import AutoTokenizer, BartForConditionalGeneration
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Hybrid Summarizer with optimized parameters and no reference dependency
class HybridSummarizer:
    def __init__(self):
        self.model_name = "facebook/bart-large-cnn"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_name)
        self.text_rank = TextRankSummarizer()

        self.best_params = {
            "max_length": 82,
            "min_length": 66,
            "num_key_sentences": 5,
            "temperature": 1.13,
            "length_penalty": 1.06
        }

    def extract_key_sentences(self, text, num_sentences=3):
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        return " ".join([str(s) for s in self.text_rank(parser.document, num_sentences)])

    def summarize(self, text):
        if not text.strip():
            return "Please enter some valid input text."

        key_points = self.extract_key_sentences(text, self.best_params["num_key_sentences"])
        prompt = f"[CNN] Generate a news summary from these key points: {key_points}"

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        if torch.cuda.is_available():
            self.model = self.model.to("cuda")
            inputs = {k: v.to("cuda") for k, v in inputs.items()}

        with torch.no_grad():
            output_ids = self.model.generate(
                inputs["input_ids"],
                num_beams=4,
                max_length=self.best_params["max_length"],
                min_length=self.best_params["min_length"],
                temperature=self.best_params["temperature"],
                length_penalty=self.best_params["length_penalty"],
                do_sample=False,
                early_stopping=True
            )

        return self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Instantiate summarizer
summarizer = HybridSummarizer()

# Gradio Interface (only input and output)
def summarize_text(text):
    return summarizer.summarize(text)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Fancy header with background gradient
    gr.HTML("""
    <div style="background: linear-gradient(to right, #667eea, #764ba2); padding: 20px; border-radius: 12px; text-align: center;">
        <h1 style="color:white;">AuroraSum Summarizer</h1>
        <p style="color:white;">Powered by BART + TextRank – Clean, Accurate, and Lightning Fast</p>
    </div>
    """)

    # Input area
    with gr.Row():
        input_box = gr.Textbox(label="📝 Input Article", lines=10, placeholder="Paste your article or paragraph here...")

    # Button and Output
    with gr.Row():
        summarize_button = gr.Button("✨ Summarize Now")

    output_box = gr.Textbox(label="📄 Generated Summary", lines=6)



    # Bind button to summarization function
    summarize_button.click(fn=summarize_text, inputs=input_box, outputs=output_box)

# Launch the app
demo.launch(share=True)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2d4afe1c25dfae4a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import nltk
nltk.download('punkt')

from transformers import AutoTokenizer, BartForConditionalGeneration
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

import evaluate
import gradio as gr

# Summarization Class
class HybridSummarizer:
    def __init__(self):
        self.model_name = "facebook/bart-large-cnn"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_name)
        self.text_rank = TextRankSummarizer()

        self.best_params = {
            "max_length": 82,
            "min_length": 66,
            "num_key_sentences": 5,
            "temperature": 1.13,
            "length_penalty": 1.06
        }

    def extract_key_sentences(self, text, num_sentences=3):
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        return " ".join(str(s) for s in self.text_rank(parser.document, num_sentences))

    def summarize(self, text):
        if not text.strip():
            return "Please enter valid input text."

        key_points = self.extract_key_sentences(text, self.best_params["num_key_sentences"])
        prompt = f"[CNN] Generate a news summary from these key points: {key_points}"

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        if torch.cuda.is_available():
            self.model.to("cuda")
            inputs = {k: v.to("cuda") for k, v in inputs.items()}

        with torch.no_grad():
            output_ids = self.model.generate(
                inputs["input_ids"],
                num_beams=4,
                max_length=self.best_params["max_length"],
                min_length=self.best_params["min_length"],
                temperature=self.best_params["temperature"],
                length_penalty=self.best_params["length_penalty"],
                do_sample=False,
                early_stopping=True
            )

        return self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Evaluation Class
class SummaryEvaluator:
    def __init__(self):
        self.rouge = evaluate.load("rouge")
        self.bertscore = evaluate.load("bertscore")
        self.meteor = evaluate.load("meteor")

    def evaluate(self, predicted, reference):
        try:
            rouge_scores = self.rouge.compute(predictions=[predicted], references=[reference])
            bert_scores = self.bertscore.compute(predictions=[predicted], references=[reference], lang="en")
            meteor_score = self.meteor.compute(predictions=[predicted], references=[reference])
        except Exception as e:
            return {"Error": f"Evaluation failed: {str(e)}"}

        return {
            "ROUGE-1": round(rouge_scores["rouge1"], 4),
            "ROUGE-2": round(rouge_scores["rouge2"], 4),
            "ROUGE-L": round(rouge_scores["rougeL"], 4),
            "METEOR": round(meteor_score["meteor"], 4),
            "BERTScore-F1": round(bert_scores["f1"][0], 4)
        }

# Initialize both
summarizer = HybridSummarizer()
evaluator = SummaryEvaluator()

# Gradio Logic
def summarize_and_evaluate(article, reference=""):
    summary = summarizer.summarize(article)
    if reference.strip():
        scores = evaluator.evaluate(summary, reference)
        score_str = "\n".join(f"{k}: {v}" for k, v in scores.items())
    else:
        score_str = "⚠️ No reference summary provided. Evaluation skipped."

    return summary, score_str

# Gradio Interface
gr.Interface(
    fn=summarize_and_evaluate,
    inputs=[
        gr.Textbox(lines=10, label="Input Article or Paragraph"),
        gr.Textbox(lines=5, label="(Optional) Reference Summary")
    ],
    outputs=[
        gr.Textbox(label="Generated Summary"),
        gr.Textbox(label="Evaluation Scores")
    ],
    title="AuroraSum: Summarization + Evaluation",
    description="Enter an article to generate a summary using BART + TextRank. Optionally add a reference summary to see evaluation scores like ROUGE, METEOR, and BERTScore."
).launch(share=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://81b54ee42eb642e94b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
